# Running a raffle with Marlowe

The raffle contract receives a deposit of a prize and then an oracle randomly selects the winner. The contract would be too large to fit on the blockchain because of the hundreds of possible winners, so Marlowe's merkleization capability is used to break up the contract into manageable pieces. The pieces are "glued together" using Marlowe's `Notify` block, so that the contract can walk a hierarchy of possible winners down to the actual winner.

Below is a diagram of the raffle contract, showing the transactions as arrows:
1.  Create the contract.
2.  Deposit the prize token.
3.  The oracle randomly chooses a winner.
4.  The first notify branches five-fold according the the random number chosen by the oracle.
    -   The *first* branch is taken if the winner is among the *first* group of 125 addresses.
    -   The *second* branch is taken if the winner is among the *second* group of 125 addresses.
    -   The *third* branch is taken if the winner is among the *third* group of 125 addresses.
    -   The *fourth* branch is taken if the winner is among the *fourth* group of 125 addresses.
    -   The *fifth* branch is taken if the winner is among the *fifth* group of 125 addresses.
5.  The second notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first group of 25 addresses within the previously selected 125 addresses.
    -   Etc.
6.  The third notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first group of 5 addresses within the previously selected 25 addresses.
    -   Etc.
7.  The fourth notify branches five-fold according the the random number chosen by the oracle.
    -   The first branch is taken if the winner is among the first of the previously selected 5 addresses, and the prize is paid to that address.
    -   Etc.
    
![Hierarchy of transactions in the raffle contract.](hierarchy.png)

## Prerequisites

#### First prize

We need to know the policy ID and token name for the NFT serves as the Marlowe role token for redeeming the first prize. There is no requirement for this token beyond that we know its policy ID and token name.

For the purposes of this narrative example, let's say we have the following. ***For the real raffle, these values must be replaced by those for the action first-prize redemption token.***

In [1]:
# Replace with actual policy and name!
PRIZE1_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE1_NAME=1stPrize

#### Second prize

Now repeat the same steps for the second prize.

For the purposes of this narrative example, let's say we have the following. For the real raffle, these values must be replaced by those for the action second-prize redemption token.

In [2]:
# Replace with actual policy and name!
PRIZE2_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE2_NAME=2ndPrize

#### Third prize

Now repeat the same steps for the third prize.

For the purposes of this narrative example, let's say we have the following. For the real raffle, these values must be replaced by those for the action third-prize redemption token.

In [3]:
# Replace with actual policy and name!
PRIZE3_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
PRIZE3_NAME=3rdPrize

## Running the raffle

### Ensure that the three prize tokens reside at the sponsor address

Send the three prize tokens (the Marlowe role tokens) to the sponsor address.

In [4]:
SPONSOR_SKEY=sponsor.skey
SPONSOR_ADDR=$(cat sponsor.mainnet.address)
echo "SPONSOR_ADDR = $SPONSOR_ADDR"

SPONSOR_ADDR = addr1q95e9feu4hkp4qwvgqasq02na05z3eg33zzjquf2d86e6qzznwng4gtlladnxm7d486psa003jy6dv230t82rvv3pflqeuzzt2


After the tokens have been sent, we can query to see them.

In [5]:
cardano-cli query utxo --mainnet --address $SPONSOR_ADDR

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
42b72c768bd5ffcc3c92adde77541f53f20f83d95b20cea6363ae6a02df42c21     0        81284961 lovelace + TxOutDatumNone
42b72c768bd5ffcc3c92adde77541f53f20f83d95b20cea6363ae6a02df42c21     1        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.3173745072697a65 + TxOutDatumNone
42b72c768bd5ffcc3c92adde77541f53f20f83d95b20cea6363ae6a02df42c21     2        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.326e645072697a65 + TxOutDatumNone
42b72c768bd5ffcc3c92adde77541f53f20f83d95b20cea6363ae6a02df42c21     3        2000000 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.3372645072697a65 + TxOutDatumNone


### Download the executables to run the raffle.

The script residing at https://github.com/input-output-hk/real-world-marlowe/tree/main/archives/raffle are compiled to executables that should work on any recent Linux machine.

In [6]:
curl -sS http://dl.marlowestat.org/raffle-bin.tar -o raffle-bin.tar

Unpack the executable files.

In [7]:
tar xvf raffle-bin.tar

bin/
bin/cardano-cli
bin/marlowe-cli
bin/marlowe-runtime-cli
bin/InitializeRaffle
bin/ExecuteRaffle


Put the executables at the front of the search path.

In [8]:
export PATH=$PWD/bin:$PATH

### Run the raffle for the first prize

First create the file that describes the role token holding the first prize.

In [9]:
cat << EOI > first-prize.token
[
    [
        "$PRIZE1_POLICY",
        "$PRIZE1_NAME"
    ]
]
EOI
cat first-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "1stPrize"
    ]
]


Now create the file with the addresses of the raffle ticket holders.

In [10]:
head first-prize.addresses

[{"asset_name":"5469636b6574233939","payment_address":"addr1q97q4e9nlhq08mrugs8pm5uq045e425w72mxhxupq9qqgykr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq3dk3ng","quantity":"1"},
 {"asset_name":"5469636b6574233938","payment_address":"addr1qxmph7666hj5yadja34pdlm8n5un0fhc9p0hv2g9ask7m49n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sq5s22j","quantity":"1"},
 {"asset_name":"5469636b6574233937","payment_address":"addr1q8pdjzhjthflmtu947feruy6rnu0zr8x7xlppgqxrew62xqwpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsaeaet6","quantity":"1"},
 {"asset_name":"5469636b6574233936","payment_address":"addr1q9zun0s6thsyf9qt0zyfnn50x8kpwutrv20eq8dgx9gt6tkr7cz4mu6gh005gdck67p7y9d8s8zsfgjkcdy75mrjh6jq2x0dua","quantity":"1"},
 {"asset_name":"5469636b6574233935","payment_address":"addr1qy5sj3v265ucpuy7ea7h7qy533xnhcl9ru3g24xcetrnxu4n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2slm3vfx","quantity":"1"},
 {"asset_name":"5469636b6574233934","payment_address":"addr1qy5lsaapah8d5pf8s2uvnnas4yk9s55vgns4jsvzz5exksgwpjujxwj

Now create the configuration file for running the raffle.

In [11]:
yaml2json << EOI > first-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: first-prize.contract
state: first-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat first-prize.config

{
    "chunkSize": 5,
    "deadlines": {
        "deposit": "3d",
        "selectWinner": "3d",
        "payout": "3d"
    },
    "runtimeURI": {
        "proxy_host": "34.70.189.123",
        "proxy_port": 3700,
        "web_host": "34.70.189.123",
        "web_port": 3780
    },
    "sponsorAddressFilePath": "sponsor.mainnet.address",
    "sponsorCollateralFilePath": "not-used",
    "sponsorPrivateKeyFilePath": "sponsor.skey",
    "contract": "first-prize.contract",
    "state": "first-prize.state",
    "tmpTxToSign": "temp.unsigned",
    "tmpTxToSubmit": "temp.signed"
}


Now build and submit the transaction that initializes the raffle.

In [12]:
InitializeRaffle first-prize.config first-prize.addresses first-prize.token

#########################
Raffle Contract Generation & Initialisation
-------------------------
 >> Raffle Contract saved in : first-prize.contract
-------------------------
Raffle Contract Generated
-------------------------
 >> Contract stored with hash: 0cfc94d494c01d82f8d94e6348afbeebf057672c985a0ddba0d0b8731e0aa3cf
Safety analysis found the following errors in the contract:
- detail: The safety analysis exceeded the allotted time.
  error: SafetyAnalysisTimeout
  fatal: false

{"contractId":"d376f1d11b5781d707e4401b625c9a088ceed201ee5bf6a905d39e212796a066#1"}
 >> tx signed
{"blockHeaderHash":"7df1fa3729cdc512cc49b10447cbcb579bab2c5b2fe008237555289e0a2907de","blockNo":9193578,"slotNo":101164664}
 >> tx submitted
 >> Contract initialzed (tx appended)
-------------------------
Raffle Contract Initialized : {"contractId":"d376f1d11b5781d707e4401b625c9a088ceed201ee5bf6a905d39e212796a066#1"}
-------------------------
#########################
{"contractId":"d376f1d11b5781d707e4401b625c9

Record the contract ID that was printed.

In [13]:
PRIZE1_CONTRACT=d376f1d11b5781d707e4401b625c9a088ceed201ee5bf6a905d39e212796a066#1

Run the raffle itself, which consists of several transactions:
1. Deposit the prize token.
2. Have the oracle retrieve a random number from https://www.random.org/.
3. Execute several Marlowe notify transactions to locate the winner.
4. Send the prize token to the winner.

In [14]:
ExecuteRaffle first-prize.config first-prize.addresses first-prize.token $PRIZE1_CONTRACT


#########################
WaitingForPrizeDeposit
 >> Depositing 1stPrize
{"txId":"f069ab54aa4ea54e660303f33307dee997ed94eb8c80423363905d69f732e845"}
 >> tx signed
{"blockHeaderHash":"d29d9a60e4fc61a19ed6ff410c844ab703faa972aa3242105a317783e08469f3","blockNo":9193582,"slotNo":101164711}
 >> tx submitted
 >> 1stPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"fa4b7a5efde77822e4a8985483260d20b7f73b13fbee5f060b465fce809baac0"}
 >> tx signed
{"blockHeaderHash":"3633be7505743fc70b9e6e3ceffc4daff389386a11dd0919752f1316d61dc01c","blockNo":9193583,"slotNo":101164749}
 >> tx submitted
>>>> Oracle has answered with 269
>>>> Winning ticket is 5469636b657423333937 (Ticket#397)
>>>> Snapshotted address is addr1qxamlpmk04dp6pwlue287kglsqk3sgq3xvz2r7cxqxnljyswpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsj25hn2
#########################

#########################
WaitingForNotify
#########################


View the contract on MarloweScan.

In [15]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE1_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=d376f1d11b5781d707e4401b625c9a088ceed201ee5bf6a905d39e212796a066%231


Record the random number from that the oracle choose, listed in the output above.

In [16]:
PRIZE1_CHOICE=269

Compute the address of the winner and view their prize on an explorer.

In [17]:
PRIZE1_WINNER=$(jq -r ".[$PRIZE1_CHOICE].payment_address" first-prize.addresses)
echo "PRIZE1_WINNER = $PRIZE1_WINNER"
echo "https://cardanoscan.io/address/$PRIZE1_WINNER"

PRIZE1_WINNER = addr1qxamlpmk04dp6pwlue287kglsqk3sgq3xvz2r7cxqxnljyswpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsj25hn2
https://cardanoscan.io/address/addr1qxamlpmk04dp6pwlue287kglsqk3sgq3xvz2r7cxqxnljyswpjujxwjglw2gelk9thqcta802lprejcu4y8dtz64uhnsj25hn2


### Run the raffle for the second prize

First create the file that describes the role token holding the second prize.

In [18]:
cat << EOI > second-prize.token
[
    [
        "$PRIZE2_POLICY",
        "$PRIZE2_NAME"
    ]
]
EOI
cat second-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "2ndPrize"
    ]
]


Now create the file for those eligible for the second prize by deleting the entry for the first prize.

In [19]:
jq "del(.[$PRIZE1_CHOICE])" first-prize.addresses > second-prize.addresses

Now create the configuration file for running the raffle.

In [20]:
yaml2json << EOI > second-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: second-prize.contract
state: second-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat second-prize.config

{
    "chunkSize": 5,
    "deadlines": {
        "deposit": "3d",
        "selectWinner": "3d",
        "payout": "3d"
    },
    "runtimeURI": {
        "proxy_host": "34.70.189.123",
        "proxy_port": 3700,
        "web_host": "34.70.189.123",
        "web_port": 3780
    },
    "sponsorAddressFilePath": "sponsor.mainnet.address",
    "sponsorCollateralFilePath": "not-used",
    "sponsorPrivateKeyFilePath": "sponsor.skey",
    "contract": "second-prize.contract",
    "state": "second-prize.state",
    "tmpTxToSign": "temp.unsigned",
    "tmpTxToSubmit": "temp.signed"
}


Now build and submit the transaction that initializes the raffle.

In [21]:
InitializeRaffle second-prize.config second-prize.addresses second-prize.token

#########################
Raffle Contract Generation & Initialisation
-------------------------
 >> Raffle Contract saved in : second-prize.contract
-------------------------
Raffle Contract Generated
-------------------------
 >> Contract stored with hash: 3fe725888131d35ca90fc2bff3293c1e74b3fc8dddbc3f7a859f6e5a4da700c7
Safety analysis found the following errors in the contract:
- detail: The safety analysis exceeded the allotted time.
  error: SafetyAnalysisTimeout
  fatal: false

{"contractId":"c7279ba01b869f38eecd8e878f5bf13cee7797d29ea8980654e3ca92e80c47f9#1"}
 >> tx signed
{"blockHeaderHash":"9607fad1cda03ffce2532f1f65e27c46f517e92325c7a95f8d310b559221707f","blockNo":9193600,"slotNo":101165018}
 >> tx submitted
 >> Contract initialzed (tx appended)
-------------------------
Raffle Contract Initialized : {"contractId":"c7279ba01b869f38eecd8e878f5bf13cee7797d29ea8980654e3ca92e80c47f9#1"}
-------------------------
#########################
{"contractId":"c7279ba01b869f38eecd8e878f5b

Record the contract ID that was printed.

In [22]:
PRIZE2_CONTRACT=c7279ba01b869f38eecd8e878f5bf13cee7797d29ea8980654e3ca92e80c47f9#1

Run the raffle itself, which consists of several transactions.

In [23]:
ExecuteRaffle second-prize.config second-prize.addresses second-prize.token $PRIZE2_CONTRACT


#########################
WaitingForPrizeDeposit
 >> Depositing 2ndPrize
{"txId":"5c7314f40423d1891c049ba30c0cbde1b7a1f0583b2d24e017adec5c81fc8d96"}
 >> tx signed
{"blockHeaderHash":"00c7fc45172856446cb77503f7af5e9d16cf6c7bd84f844418a5a426d70bf077","blockNo":9193602,"slotNo":101165069}
 >> tx submitted
 >> 2ndPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"88d9e32aebd90c444b62a45fe40f16b34642c440b217ba5f30426dc4b5c92c67"}
 >> tx signed
{"blockHeaderHash":"8772347b596d7f1f877f3f550c22119c086e2ab858104c35562e68f02ed93549","blockNo":9193604,"slotNo":101165098}
 >> tx submitted
>>>> Oracle has answered with 417
>>>> Winning ticket is 5469636b657423323632 (Ticket#262)
>>>> Snapshotted address is addr1qynqs73tdgy4g8qxm4zhxagc9ntgz4574px9de93sdw4h29n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2s5gr25w
#########################

#########################
WaitingForNotify
#########################


View the contract on MarloweScan.

In [24]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE2_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=c7279ba01b869f38eecd8e878f5bf13cee7797d29ea8980654e3ca92e80c47f9%231


Record the random number from that the oracle choose, listed in the output above.

In [25]:
PRIZE2_CHOICE=417

Compute the address of the winner and view their prize on an explorer.

In [26]:
PRIZE2_WINNER=$(jq -r ".[$PRIZE2_CHOICE].payment_address" second-prize.addresses)
echo "PRIZE2_WINNER = $PRIZE2_WINNER"
echo "https://cardanoscan.io/address/$PRIZE2_WINNER"

PRIZE2_WINNER = addr1qynqs73tdgy4g8qxm4zhxagc9ntgz4574px9de93sdw4h29n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2s5gr25w
https://cardanoscan.io/address/addr1qynqs73tdgy4g8qxm4zhxagc9ntgz4574px9de93sdw4h29n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2s5gr25w


### Run the raffle for the thrid prize

First create the file that describes the role token holding the third prize.

In [27]:
cat << EOI > third-prize.token
[
    [
        "$PRIZE3_POLICY",
        "$PRIZE3_NAME"
    ]
]
EOI
cat third-prize.token

[
    [
        "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
        "3rdPrize"
    ]
]


Now create the file for those eligible for the second prize by deleting the entry for the second prize.

In [28]:
jq "del(.[$PRIZE2_CHOICE])" second-prize.addresses > third-prize.addresses

Now create the configuration file for running the raffle.

In [29]:
yaml2json << EOI > third-prize.config
chunkSize: 5          # Danger: do not change the chunk size.
deadlines:
  deposit: 3d         # Deadline for depositing the prize token.
  selectWinner: 3d    # Deadline for selecting the winner.
  payout: 3d          # Deadline for paying the winner.
runtimeURI:
  proxy_host: 34.70.189.123  # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_HOST if a backup is needed.
  proxy_port: 3700           # The port number for Marlowe Runtime's proxy service. Use $MARLOWE_RT_PORT for the backup.
  web_host: 34.70.189.123    # The IP address of Marlowe Runtime v0.0.4. Use $MARLOWE_RT_WEBSERVER_HOST if a backup is needed.
  web_port: 3780             # The port number for Marlowe Runtime's web server. Use $MARLOWE_RT_WEBSERVER_PORT for the backup.
sponsorAddressFilePath: sponsor.mainnet.address
sponsorCollateralFilePath: not-used
sponsorPrivateKeyFilePath: sponsor.skey
contract: third-prize.contract
state: third-prize.state
tmpTxToSign: temp.unsigned
tmpTxToSubmit: temp.signed
EOI
cat third-prize.config

{
    "chunkSize": 5,
    "deadlines": {
        "deposit": "3d",
        "selectWinner": "3d",
        "payout": "3d"
    },
    "runtimeURI": {
        "proxy_host": "34.70.189.123",
        "proxy_port": 3700,
        "web_host": "34.70.189.123",
        "web_port": 3780
    },
    "sponsorAddressFilePath": "sponsor.mainnet.address",
    "sponsorCollateralFilePath": "not-used",
    "sponsorPrivateKeyFilePath": "sponsor.skey",
    "contract": "third-prize.contract",
    "state": "third-prize.state",
    "tmpTxToSign": "temp.unsigned",
    "tmpTxToSubmit": "temp.signed"
}


Now build and submit the transaction that initializes the raffle.

In [30]:
InitializeRaffle third-prize.config third-prize.addresses third-prize.token

#########################
Raffle Contract Generation & Initialisation
-------------------------
 >> Raffle Contract saved in : third-prize.contract
-------------------------
Raffle Contract Generated
-------------------------
 >> Contract stored with hash: 0da7173204450959800413ec09548839cb9c23be3f88c7963157fdcd42809a2e
Safety analysis found the following errors in the contract:
- detail: The safety analysis exceeded the allotted time.
  error: SafetyAnalysisTimeout
  fatal: false

{"contractId":"a59d76212b95f905b664b126b84df8deeb824a901ba91f684eebff62ba4a39f4#1"}
 >> tx signed
{"blockHeaderHash":"92d1d0601595faefb62f72acba1a3c70c14547deac15013bf394dc9c88bd29a7","blockNo":9193618,"slotNo":101165387}
 >> tx submitted
 >> Contract initialzed (tx appended)
-------------------------
Raffle Contract Initialized : {"contractId":"a59d76212b95f905b664b126b84df8deeb824a901ba91f684eebff62ba4a39f4#1"}
-------------------------
#########################
{"contractId":"a59d76212b95f905b664b126b84df

Record the contract ID that was printed.

In [31]:
PRIZE3_CONTRACT=a59d76212b95f905b664b126b84df8deeb824a901ba91f684eebff62ba4a39f4#1

Run the raffle itself, which consists of several transactions.

In [32]:
ExecuteRaffle third-prize.config third-prize.addresses third-prize.token $PRIZE3_CONTRACT


#########################
WaitingForPrizeDeposit
 >> Depositing 3rdPrize
{"txId":"ca2e42c3c1518f8a90ba0caf7b523cf3ed6a5527edc9a518c86cea00c76fe57d"}
 >> tx signed
{"blockHeaderHash":"eb487d06c184a63f4fac3b5edb12f7c006f1a7b3512f032cd137973f24173ab0","blockNo":9193620,"slotNo":101165427}
 >> tx submitted
 >> 3rdPrize deposited
>>>> Prize NFTs Deposited
#########################

#########################
WaitingForOracle
#########################
{"txId":"60f6ddbf070b013ddd7732a3dc146d3c233b5ae4e50f0e61aa45bb58548b3445"}
 >> tx signed
{"blockHeaderHash":"0483ebbdac44eee819ce7fb87a462a94e06fd0c8606e3f07cd23e557912e25e4","blockNo":9193622,"slotNo":101165467}
 >> tx submitted
>>>> Oracle has answered with 235
>>>> Winning ticket is 5469636b657423343237 (Ticket#427)
>>>> Snapshotted address is addr1qx0xrn7nch3w9fejlz9qh420kxmkwq9k4205ts9ey49gp39n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sy5v7gj
#########################

#########################
WaitingForNotify
#########################


View the contract on MarloweScan.

In [33]:
echo "https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=${PRIZE3_CONTRACT%%#1}%231"

https://mainnet.marlowescan.com/contractView?tab=tx-list&contractId=a59d76212b95f905b664b126b84df8deeb824a901ba91f684eebff62ba4a39f4%231


Record the random number from that the oracle choose, listed in the output above.

In [34]:
PRIZE3_CHOICE=235

Compute the address of the winner and view their prize on an explorer.

In [35]:
PRIZE3_WINNER=$(jq -r ".[$PRIZE3_CHOICE].payment_address" third-prize.addresses)
echo "PRIZE3_WINNER = $PRIZE3_WINNER"
echo "https://cardanoscan.io/address/$PRIZE3_WINNER"

PRIZE3_WINNER = addr1qx0xrn7nch3w9fejlz9qh420kxmkwq9k4205ts9ey49gp39n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sy5v7gj
https://cardanoscan.io/address/addr1qx0xrn7nch3w9fejlz9qh420kxmkwq9k4205ts9ey49gp39n3kr657fpa3q8mzwmjqvl9lqdn9g2pm3ejhlgwpprwy2sy5v7gj
